In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.core.display import display, HTML
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import pandas as pd
import argparse, logging, tempfile, json, sys, pandas, re, os

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.max_open_warning'] = 50
display(HTML("<style>.container { width:100% !important; }</style>"))

from utils import PM_HOME, GPU_NAME
sys.path.insert(0, PM_HOME)
from analysis.trace_utils import *

In [ ]:
iters = 10
model_names = ['DLRM_default', 'DLRM_MLPerf', 'resnet50', 'inception_v3']

for model_name in model_names:
    print(model_name)
    module_marker = "DLRM " if "DLRM" in model_name else "## Forward ##"
    trace_file = '{}/data/{}/e2e/{}/1.json'.format(PM_HOME, GPU_NAME, model_name)

    trimmed_trace_file = trim_trace_by_num_iter(trace_file, iters=iters)
    with open(trimmed_trace_file) as f:
        trace = json.load(f)
        
    roots, cc, corrected_start_time, corrected_end_time, sum_skipped_intervals = process_event_hierarchy(trace['traceEvents'], skip_module=False, module_marker=module_marker)
    host_runtime = corrected_end_time - corrected_start_time - sum_skipped_intervals
    device_runtime = host_runtime
    ops = []
    get_operators(roots, ops)
    
    op_device_runtime = get_device_runtime(ops, cc) # dict: op ex_id -> all its device calls and stats
    
    dt_breakdown = device_runtime_breakdown(roots, op_device_runtime, depth=0)
    # pprint(dt_breakdown)
    truncate_count = 10
    flatten = {}
    for stream, v in dt_breakdown.items():
        flatten[stream] = {}
        get_major_device_results(device_runtime, dt_breakdown[stream], flatten[stream])
    
    for stream, v in flatten.items():
        runtime_no_pf = -1
        log_file = "{}/data/{}/e2e/{}/1.log".format(PM_HOME, GPU_NAME, model_name)
        if os.path.exists(log_file):
            for line in open(log_file, 'r'):
                if re.search("Overall per-batch", line):
                    runtime_no_pf = float(line.split(' ')[4]) * 1000 * iters # us

        per_op = {}
        total = 0.0
        for k, vv in v.items():
            if k == 'total' or 'DLRM ' in k[0] or 'module' in k[0]: # Skip all labels
                continue
            k0 = k[0] if '#' not in k[0] else k[0].split('#')[0]
            if k0 not in per_op.keys():
                per_op[k0] = 0.0
            per_op[k0] += vv['runtime']
            total += vv['runtime']

        tmp = sorted(per_op.items(), key=lambda x: x[1], reverse=True)
        op = [x[0] for x in tmp]
        p = [x[1] / total for x in tmp]
        df0 = pd.DataFrame({
            'Active time': [v['total']['runtime'] / runtime_no_pf],
            'Idle time': [1 - v['total']['runtime'] / runtime_no_pf]
        })

        print(df0)

In [ ]:
# 'DLRM_default', 'DLRM_MLPerf', 'resnet50', 'inception_v3'
df = pd.DataFrame([
    [0.774223, 0.225777],
    [0.716401, 0.283599],
    [0.973628 + (0.013487 + 0.006587) * 0.9, 1 - (0.973628 + (0.013487 + 0.006587) * 0.9)], 
    [0.91137 + (0.039597 + 0.039761) * 0.9, 1 - (0.91137 + (0.039597 + 0.039761) * 0.9)]
], columns=['active', 'idle'], index=model_names)
df

In [ ]:
fontsize = 18
ax = df.plot(kind='bar', stacked=True, cmap='Set2', rot=0, fontsize=fontsize)
ax.set_xlabel("Model names", fontsize=fontsize)
ax.set_ylabel("Percentage", fontsize=fontsize)
ax.legend(bbox_to_anchor=(1, 1), fontsize=fontsize)
list_values = df['active'].tolist()
for rect, value in zip(ax.patches, list_values):
    h = rect.get_height() * 0.95
    w = rect.get_width() / 2.
    x, y = rect.get_xy()
    ax.text(x+w, y+h, "{:.2f}%".format(value*100), horizontalalignment='center', verticalalignment='center', size=fontsize)
plt.tight_layout()
plt.rcParams['figure.figsize'] = [20, 7]
plt.savefig('{}/data/{}/e2e/active_idle_split.pdf'.format(PM_HOME, GPU_NAME), bbox_inches='tight')
plt.savefig('{}/data/{}/e2e/active_idle_split.png'.format(PM_HOME, GPU_NAME), bbox_inches='tight')